In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
import matplotlib
import matplotlib.pyplot as plt
import dask
from datetime import datetime

In [2]:
# Updated for PBS scheduler
# this could go into utils.
# By default gets 1 core w/ 25 GB memory
def get_ClusterClient(ncores=1, nmem='25GB'):
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    ncores=ncores
    nmem = nmem

    cluster = PBSCluster(
        cores=ncores, # The number of cores you want
        memory=nmem, # Amount of memory
        processes=ncores, # How many processes
        queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
        resource_spec='select=1:ncpus='+str(ncores)+':mem='+nmem, # Specify resources
        project='P93300641', # Input your project ID here
        walltime='1:00:00', # Amount of wall time
        interface='ib0', # Interface to use
    )

    client = Client(cluster)
    return cluster, client

In [3]:
cluster, client = get_ClusterClient()
cluster.scale(20)

In [10]:
client.cluster

### edit the ndep files

In [28]:
f = glob.glob('/glade/scratch/djk2120/trendy2022/ndep/*.nc')
ds2 = xr.open_mfdataset(f,combine='nested',decode_times=False,parallel=True,chunks={'time':12})

In [7]:
f = '/glade/p/cgd/tss/people/dll/TRENDY2019_Forcing/Ndep/TRENDY2019_Ndep_Monthly_c190821.nc'
ds = xr.open_dataset(f)

In [30]:
ndep = xr.Dataset()
clm_names = ['dry_deposition_NHx_as_N','dry_deposition_NOy_as_N','wet_deposition_NHx_as_N','wet_deposition_NOy_as_N']
trn_names = ['drynhx','drynoy','wetnhx','wetnoy']

for clm,trn in zip(clm_names,trn_names):
    ndep[clm]=ds2[trn].copy(deep=True)
    

In [31]:
noy = 'NDEP_NOy_month'
ndep[noy] = 1000*(ds2.drynoy+ds2.wetnoy).copy(deep=True)
ndep[noy].attrs = ds[noy].attrs

nhx = 'NDEP_NHx_month'
ndep[nhx] = 1000*(ds2.drynhx+ds2.wetnhx).copy(deep=True)
ndep[nhx].attrs = ds[nhx].attrs

ntot = 'NDEP_month'
ndep[ntot] = ndep[noy]+ndep[nhx]
ndep[ntot].attrs = ds[ntot].attrs
ndep['time']=ds['time'].copy(deep=True)

In [33]:
thedir = '/glade/scratch/djk2120/trendy2022/ndep/'
nfile  = thedir+'TRENDY2022_Ndep_Monthly_c220802.nc'
ndep['dry_deposition_NHx_as_N'].to_netcdf(nfile)

In [34]:
appends = list(ndep.data_vars)
appends.remove('dry_deposition_NHx_as_N')
for thisvar in appends:
    print(thisvar)
    ndep[thisvar].to_netcdf(nfile,'a')

dry_deposition_NOy_as_N
wet_deposition_NHx_as_N
wet_deposition_NOy_as_N
NDEP_NOy_month
NDEP_NHx_month
NDEP_month


In [144]:
da = ds.datesec.copy(deep=True)
da[:]=0
ndep['datesec'] = da.copy(deep=True)

In [145]:
ndep['date']=ds['date'].copy(deep=True)

In [149]:
gw = np.cos(ndep.lat/360*2*np.pi)
gw = 2*gw/gw.sum()
ndep['gw'] = xr.DataArray(gw,dims='lat')

In [162]:
appends = ['date','datesec','gw']
for thisvar in appends:
    print(thisvar)
    ndep[thisvar].to_netcdf(nfile,'a')

date
datesec
gw
